# Домашнее задание № 2

**Создать RAG (Retrieval-Augmented Generation) чат-бота, который будет отвечать на вопросы по определённой теме.**

> RAG — это метод, который объединяет поиск по предоставленной базе знаний и генерацию текста нейросетями на их основе, что позволяет формировать осмысленный ответ.


**Для сдачи ДЗ нужно:**

1. Собрать текстовую базу знаний по любой теме. Разбить ее на фрагменты (макс. 1500 символов каждый). Пример базы данных от преподавателя.
2. Использовать код преподавателя из репозитория как шаблон. Необходимо подключить GigaChat API или Yandex Cloud AI Studio, чтобы бот не просто искал совпадения по тексту, а самостоятельно делал выводы на основе предоставленной базы знаний.
3. Подключить бота к Telegram, как показывают в лекции.

В качестве результатов работы предоставить код, вашу базу данных и скриншоты диалога с ботом.


In [89]:
GIGACHAT_CRED = "<API КЛЮЧ ДЛЯ GIGACHAT>"

TG_TOKEN = "<ТОКЕН ДЛЯ ТЕЛЕГРАМА>"

# Обработка данных


В качестве данных для базы знаний статью про Французскую Новую волну

Чтобы преобразовать pdf документ в текст будем использовать [docling](https://github.com/docling-project/docling)

In [2]:
!pip install -q docling langchain langchain-community

In [3]:
from docling.document_converter import DocumentConverter

source = "https://artculturestudies.sias.ru/upload/iblock/315/hk_2018_02_64_93_krutchkova.pdf"

# Загружаем pdf документ и преобразуем его в Markdown
converter = DocumentConverter()
result = converter.convert(source)
md_doc = result.document.export_to_markdown()

# Отображаем первые 1к символов из Markdown документа
print(md_doc[:1000])

[INFO] 2025-10-13 19:09:17,758 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-10-13 19:09:17,789 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-10-13 19:09:17,790 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-10-13 19:09:17,923 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-10-13 19:09:17,931 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-10-13 19:09:17,931 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-10-13 19:09:17,968 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-10-13 19:09:18,033 [RapidOCR] download_file.py:60: File exists and is valid: /usr/loc

КРЮЧКОВА В. А.

## Изобразительная форма в фильмах французской «новой волны». Пластика кадра пластичность сюжета

В данной статье рассматривается эстетика режиссеров французского кино Жан-Люка Годара, Жана Риветта, Эрика Ромера, Франсуа Трюффо и др. Автор обращает особое внимание на влияние Синематеки и круга журнала «Кайе дю синема» на формирование художественного вкуса, пристрастия к киноцитатам и культа визуальной достоверности у представителей «новой волны». Воссоздается атмосфера эстетического переворота и техника съемочного процесса, анализируется специфика языка и видения мира. В статье говорится о роли элементов интервью в художественной материи фильма и об установке на открытые финалы, о невозможности традиционно строить сюжет и подводить итоги ввиду незавершенности самого жизненного процесса. Метод режиссеров «новой волны» напоминает коллаж в тех случаях, когда «вырезка» (découpage) из действительности прямо внедряется в кинопоток. Но более часты иные решения: жизнь незаметно

Теперь надо получить из данного цельного текста чанки. Для этого будем использовать `RecursiveCharacterTextSplitter` из Langchain

In [33]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


CHUNK_SIZE = 1500
CHUNK_OVERLAP = 32

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([md_doc])

print(texts[0].page_content)

КРЮЧКОВА В. А.

## Изобразительная форма в фильмах французской «новой волны». Пластика кадра пластичность сюжета

В данной статье рассматривается эстетика режиссеров французского кино Жан-Люка Годара, Жана Риветта, Эрика Ромера, Франсуа Трюффо и др. Автор обращает особое внимание на влияние Синематеки и круга журнала «Кайе дю синема» на формирование художественного вкуса, пристрастия к киноцитатам и культа визуальной достоверности у представителей «новой волны». Воссоздается атмосфера эстетического переворота и техника съемочного процесса, анализируется специфика языка и видения мира. В статье говорится о роли элементов интервью в художественной материи фильма и об установке на открытые финалы, о невозможности традиционно строить сюжет и подводить итоги ввиду незавершенности самого жизненного процесса. Метод режиссеров «новой волны» напоминает коллаж в тех случаях, когда «вырезка» (découpage) из действительности прямо внедряется в кинопоток. Но более часты иные решения: жизнь незаметно

In [34]:
# Убриаем чанки - дубликаты

seen_texts = []
filtered_texts = []
for text in texts:
    page_content = text.page_content.strip()
    if page_content not in seen_texts:
        filtered_texts.append(text)
    else:
        seen_texts.append(page_content)

Теперь мы получили набор чанков, с которым можно уже работать

## Создание векторной базы данных

In [5]:
!pip install -q langchain-gigachat chroma sentence-transformers

In [38]:
from langchain_community.embeddings import HuggingFaceEmbeddings


# Загрузка модели Эмбеддинга для преобразования текста в вектор
# Ссылка на модель: https://huggingface.co/intfloat/multilingual-e5-large
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large"
)

In [48]:
emb_result = embeddings.embed_documents(
    map(lambda x: x.page_content, filtered_texts) # Получаем поля page_content у каждого документа из filtered_texts
)

In [49]:
from langchain.vectorstores.chroma import Chroma

vectorstore = Chroma(
    "vecstore",
    embeddings,
    "./chroma_db"  # папка для сохранения базы
)

In [55]:
vectorstore.add_texts(list(map(lambda x: x.page_content, filtered_texts)))

['b664d0d8-5012-4036-b34e-d6e0638d8d7c',
 '9c0eef49-e7bd-4163-9c30-4aa406fd7b73',
 '2ef4fe72-f214-4bc9-85ab-096cda580950',
 '7284157b-d98c-4af3-bf60-f31802d4cff4',
 '6236ba85-3df6-48d3-a1f5-68d60d187cc9',
 '0311a4a0-d6e1-40fd-a05e-917213671991',
 '086c0fc2-18fb-49c3-94b0-7e620861bd7e',
 '2e7483e9-6711-4cca-b2de-04324f6e492e',
 'd05d42ee-9359-417c-8292-1aaea17185d8',
 'e29b7c94-95ca-4afa-abc4-253c0105dccc',
 '4409c645-519a-4dae-a2c6-b01e7308bc5b',
 '12a369fa-8b96-47ad-bfa9-1abd5ab320bb',
 '68b29d05-1c20-4433-87b5-cb2317c436ad',
 '1b45b792-5878-4bfd-b904-18769647718a',
 '1169f111-55d3-4e6f-ad44-5c2b5aaeb30e',
 '9dd6ff43-0c06-461a-8c8a-1b8534a08fbf',
 '97c574cf-6efd-4d2b-99ac-5fb705364a05',
 '5a026355-5a48-4bc8-851b-a79e4a5bf541',
 '9aae0d92-72e9-4654-886d-c3e3d299fa23',
 'c3f162c4-45a7-4116-aaa4-e0d3d006d3e2',
 'd31138ff-cdfe-441d-a2eb-866d571b5608',
 'a88926be-f5b6-4033-943c-0f2662540603',
 '84c98378-fca3-45e9-ad20-caa3457e276b',
 '7f649f8a-eca5-42d6-b575-192a410f105d',
 'aaf35321-3c62-

In [57]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [73]:
query = "Какая роль элементов интервью?"
retrieved_chunks = retriever.invoke(query)

print("\n\n\n".join(map(lambda x: x.page_content, retrieved_chunks[0:3])))

Одна из наиболее замечательных находок - импровизированные интервью, подобные опросу людей в документальных фильмах. Чаще всего режиссер задавал вопросы актеру из-за кадра, как в «Маленьком солдате» Годара, «400 ударах» Трюффо; иной раз появлялся реальный персонаж со стороны, как в «Мужском - женском», где девушка, лицо молодежного журнала, отвечает на журналистские вопросы о своих интересах и предпочтениях. Наконец, беседа Наны, героини «Жить своей жизнью», с интеллектуалом-резонером, случайно встретившимся ей в кафе (это философ Брис Парен). Все это нарушало непрерывность фильмической ткани, создавало впечатление прорыва сквозь

нее, выхода в реальность зрителя. Отрывки из других фильмов, часто классических, с совершенно иной стилистикой, выполняли сходную роль, напоминая о том, что изображения на экране - не более чем следы событий, световые миражи, сложившиеся в картину по воле ее создателя. Эти вырезки из потока жизни, их приравнивание к фантазиям прошлым и настоящим, включение в 

Модель очень маленькая, поэтому надо брать слишком

# Создание цепочки вызовов агентов для RAG системы

In [76]:
from langchain_community.chat_models.gigachat import GigaChat

llm = GigaChat(
    credentials=GIGACHAT_CRED,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    streaming=False,
    verify_ssl_certs=False,
)

/tmp/ipython-input-2923823384.py:3: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  llm = GigaChat(


In [86]:
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from operator import itemgetter


def create_rag_chain(vectorstore, llm):
    """Создание RAG цепочки с использованием LCEL"""

    # Промпт для перефразирования
    # Данный трюк нужен, чтобы retriever (векторная бд) учитывала контекст с чатом
    # Например:
    # Пользователь спросил "какая цена акции?" -> Агент ответитил -> "А тесла?"
    # (В данном случае, перефразирование будет "Какая цена акции тесла" из чего можно найти реливантные чанки)
    rewrite_prompt = ChatPromptTemplate.from_template("""
Перефразируй вопрос для поиска в базе знаний.

История: {chat_history}
Вопрос: {question}

Перефразированный запрос:
""")

    # Промпт для ответа
    answer_prompt = ChatPromptTemplate.from_template("""
Ты -- очень начитанный и вежливый ассистент.
Твоя задача отвечать понятно пользователю, исходя из данного контекста

Контекст: {context}
Вопрос: {question}
""")

    # Функция для форматирования документов
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Функция для reranking
    def rerank_docs(inputs):
        """Переоценивает документы из векторной бд"""
        query = inputs["rewritten_query"]
        docs_with_scores = vectorstore.similarity_search_with_relevance_scores(
            query, k=10
        )
        sorted_docs = sorted(docs_with_scores, key=lambda x: x[1], reverse=True)
        return [doc for doc, score in sorted_docs[:3]]

    # Построение цепочки
    chain = (
        # Шаг 1: Перефразирование
        {
            "rewritten_query": rewrite_prompt | llm | StrOutputParser(),
            "question": itemgetter("question"),
            "chat_history": itemgetter("chat_history")
        }
        # Шаг 2 и 3: RAG + Reranking
        | RunnablePassthrough.assign(
            best_docs=RunnableLambda(rerank_docs)
        )
        # Шаг 4: Генерация ответа
        | RunnablePassthrough.assign(
            context=lambda x: format_docs(x["best_docs"])
        )
        | {
            "answer": answer_prompt | llm | StrOutputParser(),
            "rewritten_query": itemgetter("rewritten_query"),
            "best_docs": itemgetter("best_docs")
        }
    )

    return chain

chain = create_rag_chain(vectorstore, llm)
result = chain.invoke({
    "question": "О чем говорится в данной статье?",
    "chat_history": ""
})

print(result["answer"])

Статья описывает кинематографическое произведение, обращая внимание на две важные особенности:

### Пластика образов и сюжетов
Образы персонажей характеризуются плавностью контуров, будто перетекающих друг в друга, создавая необычные визуальные эффекты. Одновременно меняются смыслы отдельных сцен и целых историй, отражающие трансформационные изменения в фильме. Это подчёркивает гибкость и подвижность самого повествования, его способность переосмысливать себя по мере развития сюжета.

### Нарушение традиционного нарратива
Традиционный сюжет здесь смещён на второй план — основной акцент сделан на форме и её взаимодействиях, где элементы, такие как звук, изображение, монтаж и другие составляющие, формируют общую идею фильма. Главная идея фильма рассматривается через соотношение гармоничности или конфликта между различными элементами, составляя основу драматургии картины.

### Фильм как интеллектуальное приключение
Авторы утверждают, что фильм представляет собой именно интеллектуальное пут

## Создание телеграм бота

In [83]:
!pip install -q telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 11.4 MB/s eta 0:00:00


In [87]:
CHAT_HISTORY = []

def handle_agent_response(q: str):
    # Формируем строку с чатом
    chat_history = "\n".join(
        [
            f"\nRole: {x.get('role')}\n Content: {x.get('content')}"
            for x in CHAT_HISTORY
        ]
    )
    # Добавляем в историю чата
    CHAT_HISTORY.append({'role': "human", "content": q})

    # Возвращаем ответ из цепочки рассуждений
    return chain.invoke({
        "question": q,
        "chat_history": chat_history
    }).get('answer')

handle_agent_response("How are you doing?")

'Привет! У меня всё отлично, готов общаться и помогать тебе с любыми вопросами. Как твои дела?'

In [90]:
import telebot

bot = telebot.TeleBot(TG_TOKEN)

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    # Отправляем приветственное сообщение
    bot.send_message(message.chat.id,
                     'Привет, я бот, созданный Катей!')

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    agent_ans = handle_agent_response(message.text)
    CHAT_HISTORY.append({'role': "assistant", "content": agent_ans})

    bot.send_message(message.chat.id, agent_ans)

# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе
